In [1]:
## Convert all graphs to GR format, sort by the number of nodes

In [3]:
import os
import sys
sys.path.append('../../qtree/')
import qtree.operators as ops
import qtree.graph_model as gm_old

import networkx as nx
sys.path.append('../')
import graph_model as gm
import re
import itertools
import lzma
ENCODING = sys.getdefaultencoding()
%pdb

Automatic pdb calling has been turned ON


In [4]:
# exporting functions
def export_file(filename_in, filename_out):
    nq, circuit = ops.read_circuit_file(filename_in)
    graph_raw = gm_old.circ2graph(nq, circuit, omit_terminals=False)
    n_gates = len(list(itertools.chain.from_iterable(circuit)))
    # remove selfloops
    selfloops = list(graph_raw.selfloop_edges())
    graph_raw.remove_edges_from(selfloops)
    
    basename = filename_in.split('.')[0]
    data = gm.generate_gr_file(graph_raw)
    with lzma.open(filename_out, 'w') as fp:
        fp.write(f'c {basename}\n'.encode(ENCODING))
        fp.write(f'c number of qubits: {nq}\n'.encode(ENCODING))
        fp.write(f'c number of gates: {n_gates}\n'.encode(ENCODING))
        fp.write(data.encode(ENCODING))


def export_all(root_dir_in, root_dir_out):

    def walktree(old_top, new_top, file_callback):
        for elem in os.listdir(old_top):
            old_path = os.path.join(old_top, elem)
            print('at {}'.format(old_path))
            if os.path.isdir(old_path):
                new_path = os.path.join(new_top, elem)
                if not os.path.exists(new_path):
                    os.mkdir(new_path)
                walktree(old_path, new_path, file_callback)
            elif os.path.isfile(old_path):
                basename = elem.split('.')[0]
                suffix = elem.split('.')[-1]
                if suffix == 'txt':
                    new_path = os.path.join(new_top, basename+'.gr.xz')
                    file_callback(old_path, new_path)

    walktree(root_dir_in, root_dir_out, export_file)

In [5]:
def find_n_nodes(filename):
    # search for the header line
    comment_patt = re.compile('^(\s*c\s+)(?P<comment>.*)'.encode(ENCODING))
    header_patt = re.compile(
        '^(\s*p\s+)((?P<file_type>cnf|tw)\s+)?(?P<n_nodes>\d+)\s+(?P<n_edges>\d+)'.encode(ENCODING)
    )
    with lzma.open(filename, 'r') as fp:
        for n, line in enumerate(fp):
            m = re.search(comment_patt, line)
            if m is not None:
                continue
            m = re.search(header_patt, line)
            if m is not None:
                n_nodes = int(m.group('n_nodes'))
                n_edges = int(m.group('n_edges'))
                break
            else:
                raise ValueError(f'File format error at line {n}:\n'
                             f' expected pattern: {header_patt}')
    return n_nodes

def sort_by_n_nodes(root_dir_in):

    files_by_nodes = dict()
    def walktree(top, file_callback):
        for elem in os.listdir(top):
            path = os.path.join(top, elem)
            print('at {}'.format(path))
            if os.path.isdir(path):
                walktree(path, file_callback)
            elif os.path.isfile(path):
                basename = elem.split('.')[0]
                suffix = elem.split('.')[-1]
                if suffix == 'xz':
                    n_nodes = file_callback(path)
                    if n_nodes in files_by_nodes:
                        files_by_nodes[n_nodes].append(path)
                    else:
                        files_by_nodes[n_nodes] = [path]
    
    walktree(root_dir_in, find_n_nodes)
    return files_by_nodes

In [6]:
# export all files from circuits to GR format
# export_all('../../qtree/test_circuits', './boixo_circuits')

In [7]:
os.chdir('../')
os.getcwd()

'/Users/roman/Documents/projects/q_simulator/graph-rl'

In [10]:
# find how many nodes each file has and build an index
files_by_nodes = sort_by_n_nodes('./graph_test/boixo_circuits')
f_by_nodes = {key: tuple(val) for key, val in files_by_nodes.items()}

import pickle
with open('boixo_circuits_index.p', 'wb') as fp:
    pickle.dump(f_by_nodes, fp)

at ./graph_test/boixo_circuits/index.p
at ./graph_test/boixo_circuits/inst
at ./graph_test/boixo_circuits/inst/cz_v2
at ./graph_test/boixo_circuits/inst/cz_v2/6x6
at ./graph_test/boixo_circuits/inst/cz_v2/6x6/inst_6x6_37_5.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/6x6/inst_6x6_27_5.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/6x6/inst_6x6_34_4.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/6x6/inst_6x6_24_4.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/6x6/inst_6x6_19_1.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/6x6/inst_6x6_11_7.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/6x6/inst_6x6_50_9.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/6x6/inst_6x6_40_9.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/6x6/inst_6x6_12_6.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/6x6/inst_6x6_53_8.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/6x6/inst_6x6_43_8.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/6x6/inst_6x6_31_8.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/6x6

at ./graph_test/boixo_circuits/inst/cz_v2/6x6/inst_6x6_75_1.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/6x6/inst_6x6_40_2.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/6x6/inst_6x6_48_4.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/6x6/inst_6x6_50_2.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/6x6/inst_6x6_58_4.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/6x6/inst_6x6_43_3.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/6x6/inst_6x6_53_3.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/6x6/inst_6x6_65_3.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/6x6/inst_6x6_75_3.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/6x6/inst_6x6_66_2.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/6x6/inst_6x6_76_2.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/6x6/inst_6x6_43_1.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/6x6/inst_6x6_53_1.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/6x6/inst_6x6_40_0.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/6x6/inst_6x6_48_6.gr.xz
at ./graph

at ./graph_test/boixo_circuits/inst/cz_v2/4x5/inst_4x5_27_1.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/4x5/inst_4x5_34_0.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/4x5/inst_4x5_24_0.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/4x5/inst_4x5_11_3.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/4x5/inst_4x5_19_5.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/4x5/inst_4x5_12_2.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/4x5/inst_4x5_70_2.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/4x5/inst_4x5_78_4.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/4x5/inst_4x5_60_2.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/4x5/inst_4x5_68_4.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/4x5/inst_4x5_73_3.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/4x5/inst_4x5_63_3.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/4x5/inst_4x5_56_0.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/4x5/inst_4x5_46_0.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/4x5/inst_4x5_55_1.gr.xz
at ./graph

at ./graph_test/boixo_circuits/inst/cz_v2/4x5/inst_4x5_23_1.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/4x5/inst_4x5_59_7.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/4x5/inst_4x5_51_1.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/4x5/inst_4x5_49_7.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/4x5/inst_4x5_41_1.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/4x5/inst_4x5_18_9.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/4x5/inst_4x5_52_0.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/4x5/inst_4x5_42_0.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/4x5/inst_4x5_77_3.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/4x5/inst_4x5_67_3.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/4x5/inst_4x5_74_2.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/4x5/inst_4x5_64_2.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/8x9
at ./graph_test/boixo_circuits/inst/cz_v2/8x9/inst_8x9_42_2.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/8x9/inst_8x9_52_2.gr.xz
at ./graph_test/boixo_circuits

at ./graph_test/boixo_circuits/inst/cz_v2/8x9/inst_8x9_42_7.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/8x9/inst_8x9_52_7.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/8x9/inst_8x9_13_9.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/8x9/inst_8x9_67_4.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/8x9/inst_8x9_77_4.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/8x9/inst_8x9_64_5.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/8x9/inst_8x9_74_5.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/8x9/inst_8x9_42_5.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/8x9/inst_8x9_52_5.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/8x9/inst_8x9_41_4.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/8x9/inst_8x9_49_2.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/8x9/inst_8x9_51_4.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/8x9/inst_8x9_59_2.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/8x9/inst_8x9_64_7.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/8x9/inst_8x9_74_7.gr.xz
at ./graph

...

at ./graph_test/boixo_circuits/inst/cz_v2/9x9/inst_9x9_26_1.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/9x9/inst_9x9_36_1.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/9x9/inst_9x9_13_2.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/9x9/inst_9x9_10_3.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/9x9/inst_9x9_18_5.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/9x9/inst_9x9_62_3.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/9x9/inst_9x9_72_3.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/9x9/inst_9x9_61_2.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/9x9/inst_9x9_69_4.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/9x9/inst_9x9_71_2.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/9x9/inst_9x9_79_4.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/9x9/inst_9x9_44_1.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/9x9/inst_9x9_54_1.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/9x9/inst_9x9_47_0.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/9x9/inst_9x9_57_0.gr.xz
at ./graph

at ./graph_test/boixo_circuits/inst/cz_v2/7x7/inst_7x7_64_4.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/7x7/inst_7x7_51_7.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/7x7/inst_7x7_59_1.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/7x7/inst_7x7_41_7.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/7x7/inst_7x7_49_1.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/7x7/inst_7x7_10_9.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/7x7/inst_7x7_52_6.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/7x7/inst_7x7_42_6.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/7x7/inst_7x7_13_8.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/7x7/inst_7x7_15_1.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/7x7/inst_7x7_16_0.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/7x7/inst_7x7_33_3.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/7x7/inst_7x7_23_3.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/7x7/inst_7x7_30_2.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/7x7/inst_7x7_38_4.gr.xz
at ./graph

at ./graph_test/boixo_circuits/inst/cz_v2/7x7/inst_7x7_72_4.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/7x7/inst_7x7_18_2.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/7x7/inst_7x7_10_4.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/7x7/inst_7x7_13_5.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/7x7/inst_7x7_67_8.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/7x7/inst_7x7_77_8.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/7x7/inst_7x7_26_6.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/7x7/inst_7x7_36_6.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/7x7/inst_7x7_64_9.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/7x7/inst_7x7_74_9.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/7x7/inst_7x7_25_7.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/7x7/inst_7x7_35_7.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/7x7/inst_7x7_55_2.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/7x7/inst_7x7_80_9.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/7x7/inst_7x7_45_2.gr.xz
at ./graph

at ./graph_test/boixo_circuits/inst/cz_v2/10x11/inst_10x11_36_6.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/10x11/inst_10x11_26_6.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/10x11/inst_10x11_77_8.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/10x11/inst_10x11_67_8.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/10x11/inst_10x11_35_7.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/10x11/inst_10x11_25_7.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/10x11/inst_10x11_74_9.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/10x11/inst_10x11_64_9.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/10x11/inst_10x11_16_9.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/10x11/inst_10x11_57_7.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/10x11/inst_10x11_47_7.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/10x11/inst_10x11_15_8.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/10x11/inst_10x11_54_6.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/10x11/inst_10x11_44_6.gr.xz
at ./graph_test/boix

at ./graph_test/boixo_circuits/inst/cz_v2/10x11/inst_10x11_77_1.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/10x11/inst_10x11_15_1.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/10x11/inst_10x11_16_0.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/10x11/inst_10x11_23_3.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/10x11/inst_10x11_33_3.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/10x11/inst_10x11_28_4.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/10x11/inst_10x11_20_2.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/10x11/inst_10x11_38_4.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/10x11/inst_10x11_30_2.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/10x11/inst_10x11_50_7.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/10x11/inst_10x11_58_1.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/10x11/inst_10x11_40_7.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/10x11/inst_10x11_48_1.gr.xz
at ./graph_test/boixo_circuits/inst/cz_v2/10x11/inst_10x11_11_9.gr.xz
at ./graph_test/boix

In [11]:
with open('boixo_circuits_index.p', 'rb') as fp:
    f_by_nodes = pickle.load(fp)

In [12]:
# build index for pace2017 main instances
# find how many nodes each file has and build an index
files_by_nodes = sort_by_n_nodes('./graph_test/pace2017_instances/gr')
f_by_nodes = {key: tuple(val) for key, val in files_by_nodes.items()}

import pickle
with open('pace2017_instances_index.p', 'wb') as fp:
    pickle.dump(f_by_nodes, fp)

at ./graph_test/pace2017_instances/gr/heuristic
at ./graph_test/pace2017_instances/gr/heuristic/he117.gr.xz
at ./graph_test/pace2017_instances/gr/heuristic/he152.gr.xz
at ./graph_test/pace2017_instances/gr/heuristic/he072.gr.xz
at ./graph_test/pace2017_instances/gr/heuristic/he037.gr.xz
at ./graph_test/pace2017_instances/gr/heuristic/he192.gr.xz
at ./graph_test/pace2017_instances/gr/heuristic/he056.gr.xz
at ./graph_test/pace2017_instances/gr/heuristic/he013.gr.xz
at ./graph_test/pace2017_instances/gr/heuristic/he133.gr.xz
at ./graph_test/pace2017_instances/gr/heuristic/he176.gr.xz
at ./graph_test/pace2017_instances/gr/heuristic/he096.gr.xz
at ./graph_test/pace2017_instances/gr/heuristic/he131.gr.xz
at ./graph_test/pace2017_instances/gr/heuristic/he174.gr.xz
at ./graph_test/pace2017_instances/gr/heuristic/he189.gr.xz
at ./graph_test/pace2017_instances/gr/heuristic/he094.gr.xz
at ./graph_test/pace2017_instances/gr/heuristic/he069.gr.xz
at ./graph_test/pace2017_instances/gr/heuristic/he05

at ./graph_test/pace2017_instances/gr/exact/ex027.gr.xz
at ./graph_test/pace2017_instances/gr/exact/ex046.gr.xz
at ./graph_test/pace2017_instances/gr/exact/ex003.gr.xz
at ./graph_test/pace2017_instances/gr/exact/ex086.gr.xz
at ./graph_test/pace2017_instances/gr/exact/ex123.gr.xz
at ./graph_test/pace2017_instances/gr/exact/ex166.gr.xz
at ./graph_test/pace2017_instances/gr/exact/ex084.gr.xz
at ./graph_test/pace2017_instances/gr/exact/ex079.gr.xz
at ./graph_test/pace2017_instances/gr/exact/ex121.gr.xz
at ./graph_test/pace2017_instances/gr/exact/ex164.gr.xz
at ./graph_test/pace2017_instances/gr/exact/ex199.gr.xz
at ./graph_test/pace2017_instances/gr/exact/ex159.gr.xz
at ./graph_test/pace2017_instances/gr/exact/ex044.gr.xz
at ./graph_test/pace2017_instances/gr/exact/ex001.gr.xz
at ./graph_test/pace2017_instances/gr/exact/ex138.gr.xz
at ./graph_test/pace2017_instances/gr/exact/ex180.gr.xz
at ./graph_test/pace2017_instances/gr/exact/ex060.gr.xz
at ./graph_test/pace2017_instances/gr/exact/ex02

In [28]:
# build index for pace2017 bonus instances
# find how many nodes each file has and build an index
files_by_nodes = sort_by_n_nodes('graph_test/pace2017_bonus_instances/gr')
f_by_nodes = {key: tuple(val) for key, val in files_by_nodes.items()}

import pickle
with open('pace2017_bonus_instances_index.p', 'wb') as fp:
    pickle.dump(f_by_nodes, fp)

at graph_test/pace2017_bonus_instances/gr/Promedas_61_8.gr.xz
at graph_test/pace2017_bonus_instances/gr/Promedas_62_9.gr.xz
at graph_test/pace2017_bonus_instances/gr/NY_11.gr.xz
at graph_test/pace2017_bonus_instances/gr/Pedigree_11_7.gr.xz
at graph_test/pace2017_bonus_instances/gr/Promedas_48_5.gr.xz
at graph_test/pace2017_bonus_instances/gr/SAT_dat.k80-24_1_rule_1.gaifman_3.gr.xz
at graph_test/pace2017_bonus_instances/gr/jgiraldezlevy.2200.9086.08.40.46.gaifman_2.gr.xz
at graph_test/pace2017_bonus_instances/gr/Promedus_14_9.gr.xz
at graph_test/pace2017_bonus_instances/gr/Promedus_18_10.gr.xz
at graph_test/pace2017_bonus_instances/gr/NY_13.gr.xz
at graph_test/pace2017_bonus_instances/gr/Promedas_25_8.gr.xz
at graph_test/pace2017_bonus_instances/gr/Promedas_44_9.gr.xz
at graph_test/pace2017_bonus_instances/gr/Promedas_46_15.gr.xz
at graph_test/pace2017_bonus_instances/gr/Sz512_15127_1.smt2-stp212.gaifman_3.gr.xz
at graph_test/pace2017_bonus_instances/gr/Promedas_34_8.gr.xz
at graph_test